In [1]:
# Load and read PDF content
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("pdf/DCPR_2034_13-09-2024.pdf")
docs = loader.load()
print("Number of pages in the PDF:", len(docs))

# Extract text from each page for embedding
doc_texts = [page.page_content for page in docs]
#print(" Document  texts",doc_texts)


Number of pages in the PDF: 613


In [2]:
from milvus_model.hybrid import BGEM3EmbeddingFunction
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
ef = BGEM3EmbeddingFunction(use_fp16=False, device=device)
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(doc_texts)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 39/39 [17:51<00:00, 27.47s/it]


In [3]:
dense_dim

1024

In [4]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [5]:
# Connect to Milvus given URI
try:
    connections.connect(
        alias="default",
        uri="https://in03-4e569f605c32eab.serverless.gcp-us-west1.cloud.zilliz.com",  # Replace with your endpoint
        token="99cd003de770782d436a049c87fb669188dc4424443531a325043d7f42859ca8c3d058b952d2e92d33677cf72b4931d12150c29d"  # Replace with your API key
    )
    print("Successfully connected to Milvus")
except Exception as e:
    print(f"Failed to connect to Milvus: {e}")


Successfully connected to Milvus


In [6]:
# Specify the data schema for the new Collection
fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]

# Define the collection name
collection_name = "hybrid_demo"
schema = CollectionSchema(fields)

# Create collection
col = Collection(
    name=collection_name, 
    schema=schema, 
    using='default',  # Uses the default connection,
    consistency_level="Strong"
)

# To make vector search efficient, we need to create indices for the vector fields
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()

In [7]:




# For efficiency, we insert 50 records in each small batch
for i in range(0, len(doc_texts), 50):
    # Extract the raw text from Document objects
    batched_texts = [doc.page_content for doc in docs[i : i + 50]]  # Extract the text content
    batched_sparse_vectors = docs_embeddings["sparse"][i : i + 50]
    batched_dense_vectors = docs_embeddings["dense"][i : i + 50]

    # Prepare the batched entities for insertion
    batched_entities = [
        batched_texts,          # Text content
        batched_sparse_vectors, # Sparse vector embeddings
        batched_dense_vectors,  # Dense vector embeddings
    ]

    # Insert the batch into Milvus
    col.insert(batched_entities)

print("Number of entities inserted:", col.num_entities)

Number of entities inserted: 1226


In [22]:
# Enter your search query
query = input("Enter your search query: ")
print(query)

# Generate embeddings for the query
query_embeddings = ef([query])
#print(query_embeddings)

Incentive in 33(7)


In [23]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)

def dense_search(col, query_dense_embedding, limit=5):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [24]:
def sparse_search(col, query_sparse_embedding, limit=5):
    search_params = {
        "metric_type": "IP", "params": {}, }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [25]:
def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=5,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]

In [26]:
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"][[0]])
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"][[0]],
    sparse_weight=0.7,
    dense_weight=1.0,
)

In [27]:
print(dense_results)

['Original Notifications shall\nbe referred for perspicuity\n33(7)(B) Additional FSI for Redevelopment of existing\nresidential housing societies excluding buildings covered\nunder regulation 33(7) and 33(7)(A):\n1) In case of redevelopment of existing residential housing societies\nexcluding buildings covered under regulation 33(7) and 33(7)(A)\nproposed by Housing societies/landlords or through their proponents\nwhere existing members are proposed to be re- accommodated on the\nsame plot, incentive additional BUA to the extent of 15% of existing\nBUA or 10 sq. m per tenement whichever is more shall be permissible\nwithout premium.\nProvided further that if the existing authorized BUA and incentive\nthereon as stated above is less than the permissible FSI as per\nregulation 30(A)(1), then society may avail ‘Additional FSI on payment\nof premium/TDR’ up to limit of permissible FSI.\n2) If staircase, lift & lift lobby areas are claimed free of FSI by charging\npremium as per then prevai

In [28]:
print(sparse_results)

['Original Notifications shall\nbe referred for perspicuity\nredevelopment.\n16. Restriction on transfer of tenements shall be governed by provisions\nof Maharashtra Rent Control Act, till such time that a Co-op. Housing Society\nis formed and thereafter the same shall be governed by the provision of\nMaharashtra Co-op. Societies Act. However, tenements constructed for slum\nrehabilitation shall not be transferable for a period of 10 years.\n17. Corpus Fund: An amount of minimum Rs. 50,000/- per tenement or as\ndirected by the HPC shall be created by the Promoter/Developer as a\nCorpus fund, which will be utilised for maintenance of the\nrehabilitation buildings for a period of 10 years.\n18. Any ongoing scheme under Regulation 33(7) which fulfills the criteria\nunder this Regulation can be included in the proposal under Regulation\n33(9) for approval or converted into a CDS under this Regulation 33(9).\nHowever, all dilutions of reservations under Regulation 33(7) shall have\nto be re

In [29]:
print(hybrid_results)

['Original Notifications shall\nbe referred for perspicuity\n33(7)(B) Additional FSI for Redevelopment of existing\nresidential housing societies excluding buildings covered\nunder regulation 33(7) and 33(7)(A):\n1) In case of redevelopment of existing residential housing societies\nexcluding buildings covered under regulation 33(7) and 33(7)(A)\nproposed by Housing societies/landlords or through their proponents\nwhere existing members are proposed to be re- accommodated on the\nsame plot, incentive additional BUA to the extent of 15% of existing\nBUA or 10 sq. m per tenement whichever is more shall be permissible\nwithout premium.\nProvided further that if the existing authorized BUA and incentive\nthereon as stated above is less than the permissible FSI as per\nregulation 30(A)(1), then society may avail ‘Additional FSI on payment\nof premium/TDR’ up to limit of permissible FSI.\n2) If staircase, lift & lift lobby areas are claimed free of FSI by charging\npremium as per then prevai

In [30]:
def doc_text_formatting(ef, query, docs):
    tokenizer = ef.model.tokenizer
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close:
                    formatted_text += "</span>"
                else:
                    formatted_text += "<span style='color:red'>"
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "</span>"
        formatted_texts.append(formatted_text)
    return formatted_texts

In [31]:
from IPython.display import Markdown, display

# Dense search results
display(Markdown("**Dense Search Results:**"))
formatted_results = doc_text_formatting(ef, query, dense_results)
for result in dense_results:
    display(Markdown(result))

**Dense Search Results:**

Original Notifications shall
be referred for perspicuity
33(7)(B) Additional FSI for Redevelopment of existing
residential housing societies excluding buildings covered
under regulation 33(7) and 33(7)(A):
1) In case of redevelopment of existing residential housing societies
excluding buildings covered under regulation 33(7) and 33(7)(A)
proposed by Housing societies/landlords or through their proponents
where existing members are proposed to be re- accommodated on the
same plot, incentive additional BUA to the extent of 15% of existing
BUA or 10 sq. m per tenement whichever is more shall be permissible
without premium.
Provided further that if the existing authorized BUA and incentive
thereon as stated above is less than the permissible FSI as per
regulation 30(A)(1), then society may avail ‘Additional FSI on payment
of premium/TDR’ up to limit of permissible FSI.
2) If staircase, lift & lift lobby areas are claimed free of FSI by charging
premium as per then prevailing Regulation, then such areas to that
extent only will be granted free of FSI without charging premium. If
staircase, lift & lift lobby areas are counted in FSI in earlier
development, then incentive additional FSI as stated in Sr. No. 1 shall
also be given on such area & such areas may be availed free of FSI by
charging premium as per these Regulations.
3) This Regulation shall be applicable only when existing members of the
societies are proposed to be re-accommodated.
4) This regulation will be applicable for redevelopment of existing
authorized buildings which are of thirty years of age or more.
5) This regulation shall not be applicable in respect of redevelopment
proposal to be/being processed under Regulation No. 33(5), 33(7),
33(8), 33(9), 33(9)(A), 33(9)(B), 33(10), 33(10) (A), 33(20) (A), 33(21).
Explanation: -Age of a building shall be as on the 1st of January of the
year in which a complete redevelopment proposal is submitted to the
Commissioner and shall be calculated from the date of Occupation
Certificate or alternately, from the first date of assessment as per the
property tax record in respect of such building or building on which
balance/admissible FSI has been consumed by way of vertical
extension as per then prevailing Regulation, available with the MCGM.
6) This incentive additional BUA shall be independent of additional BUA
as permissible under Regulation No. 14(A), 15, 16 and 17, if any.
7) Fungible compensatory area admissible under Regulation No. 31(3) on
the existing authorised BUA shall be without charging premium and
over the incentive additional BUA by charging of premium.
8) If tenanted building/s and building/s of co-operative housing
society/non- tenanted building/s co-exist on the plot under
development, then proportionate land component as per the existing
authorised BUA of existing tenanted building on the plot shall be
180 DCPR 2034


Original Notifications shall
be referred for perspicuity
33(7)(B) Additional FSI for Redevelopment of existing
residential housing societies excluding buildings covered
under regulation 33(7) and 33(7)(A):
1) In case of redevelopment of existing residential housing societies
excluding buildings covered under regulation 33(7) and 33(7)(A)
proposed by Housing societies/landlords or through their proponents
where existing members are proposed to be re- accommodated on the
same plot, incentive additional BUA to the extent of 15% of existing
BUA or 10 sq. m per tenement whichever is more shall be permissible
without premium.
Provided further that if the existing authorized BUA and incentive
thereon as stated above is less than the permissible FSI as per
regulation 30(A)(1), then society may avail ‘Additional FSI on payment
of premium/TDR’ up to limit of permissible FSI.
2) If staircase, lift & lift lobby areas are claimed free of FSI by charging
premium as per then prevailing Regulation, then such areas to that
extent only will be granted free of FSI without charging premium. If
staircase, lift & lift lobby areas are counted in FSI in earlier
development, then incentive additional FSI as stated in Sr. No. 1 shall
also be given on such area & such areas may be availed free of FSI by
charging premium as per these Regulations.
3) This Regulation shall be applicable only when existing members of the
societies are proposed to be re-accommodated.
4) This regulation will be applicable for redevelopment of existing
authorized buildings which are of thirty years of age or more.
5) This regulation shall not be applicable in respect of redevelopment
proposal to be/being processed under Regulation No. 33(5), 33(7),
33(8), 33(9), 33(9)(A), 33(9)(B), 33(10), 33(10) (A), 33(20) (A), 33(21).
Explanation: -Age of a building shall be as on the 1st of January of the
year in which a complete redevelopment proposal is submitted to the
Commissioner and shall be calculated from the date of Occupation
Certificate or alternately, from the first date of assessment as per the
property tax record in respect of such building or building on which
balance/admissible FSI has been consumed by way of vertical
extension as per then prevailing Regulation, available with the MCGM.
6) This incentive additional BUA shall be independent of additional BUA
as permissible under Regulation No. 14(A), 15, 16 and 17, if any.
7) Fungible compensatory area admissible under Regulation No. 31(3) on
the existing authorised BUA shall be without charging premium and
over the incentive additional BUA by charging of premium.
8) If tenanted building/s and building/s of co-operative housing
society/non- tenanted building/s co-exist on the plot under
development, then proportionate land component as per the existing
authorised BUA of existing tenanted building on the plot shall be
180 DCPR 2034


Original Notifications shall
be referred for perspicuity
33(7)(B) Additional FSI for Redevelopment of existing
residential housing societies excluding buildings covered
under regulation 33(7) and 33(7)(A):
1) In case of redevelopment of existing residential housing societies
excluding buildings covered under regulation 33(7) and 33(7)(A)
proposed by Housing societies/landlords or through their proponents
where existing members are proposed to be re- accommodated on the
same plot, incentive additional BUA to the extent of 15% of existing
BUA or 10 sq. m per tenement whichever is more shall be permissible
without premium.
Provided further that if the existing authorized BUA and incentive
thereon as stated above is less than the permissible FSI as per
regulation 30(A)(1), then society may avail ‘Additional FSI on payment
of premium/TDR’ up to limit of permissible FSI.
2) If staircase, lift & lift lobby areas are claimed free of FSI by charging
premium as per then prevailing Regulation, then such areas to that
extent only will be granted free of FSI without charging premium. If
staircase, lift & lift lobby areas are counted in FSI in earlier
development, then incentive additional FSI as stated in Sr. No. 1 shall
also be given on such area & such areas may be availed free of FSI by
charging premium as per these Regulations.
3) This Regulation shall be applicable only when existing members of the
societies are proposed to be re-accommodated.
4) This regulation will be applicable for redevelopment of existing
authorized buildings which are of thirty years of age or more.
5) This regulation shall not be applicable in respect of redevelopment
proposal to be/being processed under Regulation No. 33(5), 33(7),
33(8), 33(9), 33(9)(A), 33(9)(B), 33(10), 33(10) (A), 33(20) (A), 33(21).
Explanation: -Age of a building shall be as on the 1st of January of the
year in which a complete redevelopment proposal is submitted to the
Commissioner and shall be calculated from the date of Occupation
Certificate or alternately, from the first date of assessment as per the
property tax record in respect of such building or building on which
balance/admissible FSI has been consumed by way of vertical
extension as per then prevailing Regulation, available with the MCGM.
6) This incentive additional BUA shall be independent of additional BUA
as permissible under Regulation No. 14(A), 15, 16 and 17, if any.
7) Fungible compensatory area admissible under Regulation No. 31(3) on
the existing authorised BUA shall be without charging premium and
over the incentive additional BUA by charging of premium.
8) If tenanted building/s and building/s of co-operative housing
society/non- tenanted building/s co-exist on the plot under
development, then proportionate land component as per the existing
authorised BUA of existing tenanted building on the plot shall be
180 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
bb.. BByy aappppooiinnttiinngg PPrriivvaattee ddeevveellooppeerr bbyy aauutthhoorriittyy wwiitthh ttrriippaarrttyy aaggrreeeemmeenntt
bbeettwweeeenn ddeevveellooppeerr MMHHAADDAA//MMCCGGMM aauutthhoorriittyy aanndd ooccccuuppaannttss 7. FSI Permissible:
aassssoocciiaattiioonn aass ppeerr 77((1133)) ooff tthhiiss rreegguullaattiioonn.. A. For redevelopment undertaken by Private developers as per clause 2(a)
cc.. MMHHAADDAA // MMCCGGMM tthheemmsseellvveess bbyy aappppooiinnttiinngg ddeevveellooppeerr tthhrroouugghh above:
ccoommppeettiittiivvee bbiiddddiinngg aass ppeerr ccllaauussee 77((CC)) ooff tthhiiss rreegguullaattiioonn FSI required for rehabilitation of existing occupants plus incentive
33.. AAllll tthhee eelliiggiibbllee ooccccuuppaannttss iinn bbuuiillddiinngg//ssttrruuccttuurreess sshhaallll bbee cceerrttiiffiieedd bbyy thereon as mentioned below, subject to minimum 3 FSI
rreessppeeccttiivvee aauutthhoorriittiieess oorr MMHHAADDAA // MMCCGGMM..
44.. EEaacchh ooccccuuppaanntt sshhaallll bbee rreehhaabbiilliittaatteedd aanndd ggiivveenn tthhee ccaarrppeett aarreeaa ooccccuuppiieedd Basic Ratio LR/RC Incentive (as % of Admissible rehab Area)
bbyy hhiimm ffoorr rreessiiddeennttiiaall ppuurrppoossee iinn tthhee oolldd bbuuiillddiinngg ssuubbjjeecctt ttoo tthhee For Single Plot For Composite redevelopment of
mmiinniimmuumm ffiixxeedd ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq..fftt..)) aanndd//oorr mmaaxxiimmuumm Two to Five Plots Six or more Plots
ccaarrppeett aarreeaa uuppttoo 112200 ssqq..mm ((11229922 ssqq..fftt..)) IInn ccaassee ooff nnoonn--rreessiiddeennttiiaall Above 6 75 85 90
ooccccuuppiieerr,, tthhee aarreeaa ttoo bbee ggiivveenn iinn tthhee rreeccoonnssttrruucctteedd bbuuiillddiinngg wwiillll bbee Above 4 and upto 6 78 88 95
eeqquuiivvaalleenntt ttoo tthhee aarreeaa ooccccuuppiieedd iinn tthhee oolldd bbuuiillddiinngg.. PPrroovviiddeedd ffuurrtthheerr tthhaatt Upto 4 80 90 100
eeaacchh eelliiggiibbllee rreessiiddeennttiiaall ccuumm ccoommmmeerrcciiaall ooccccuuppaanntt sshhaallll bbee eennttiittlleedd ttoo
aa tteenneemmeenntt ooff mmiinniimmuumm ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq.. mm..))..
Explanation: - RC is rate of construction in respect of RCC
PPrroovviiddeedd ffuurrtthheerr tthhaatt tthhee rreehhaabbiilliittaattiioonn aarreeaa ooff aannyy ooccccuuppaanntt ooff aa
Construction and Land Rate (LR) is the rate of Open Land for FSI 1.
ccoommmmeerrcciiaall eessttaabblliisshhmmeenntt,, wwhhoo iiss aalllloowweedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy
Provided further that in ease there is more than one land rate applicable
MMHHAADDAA//MMCCGGMM ttoo bbee rreehhaabbiilliittaatteedd iinn aa rreessiiddeennttiiaall tteenneemmeenntt iinn lliieeuu
to different parts of the plot under the CDS, a weighted average of all
ooff hhiiss ccoommmmeerrcciiaall eessttaabblliisshhmmeenntt,, sshhaallll aallssoo bbee ggoovveerrnneedd bbyy tthhee
the applicable rates shall be taken for calculating the Average Land Rate
aaffoorreessaaiidd pprroovviissiioonnss aapppplliiccaabbllee ttoo tthhee rreessiiddeennttiiaall ooccccuuppaannttss iinn tthheessee
and the Basic Ratio. Provided further that for calculation of the Basic
RReegguullaattiioonnss..
Ratio, the Land Rate (LR) and the Rate of Construction (RC) shall be
55.. TThhee lliisstt ooff eelliiggiibbllee ooccccuuppaannttss aanndd aarreeaa ooccccuuppiieedd bbyy eeaacchh ooff tthheemm sshhaallll
taken for the year in which the proposal is approved by the Authority
bbee cceerrttiiffiieedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy ii..ee.. MMHHAADDAA oorr MMCCGGMM..
and the said ratio shall remain unchanged even if such Scheme
66.. FFuunnggiibbllee CCoommppeennssaattoorryy AArreeaa aass aapppplliiccaabbllee oonn tthhee aarreeaa ttoo bbee hhaannddeedd
undergoes any revision or modification subsequently during its course
oovveerr ttoo MMHHAADDAA//MMCCGGMM sshhaallll nnoott bbee aalllloowweedd ttoo bbee uuttiilliizzeedd oonn ssaallee
of completion.
ccoommppoonneenntt.. NNoo pprreemmiiuumm sshhaallll bbee cchhaarrggeedd oonn tthhee ffuunnggiibbllee
ccoommppeennssaattoorryy aarreeaa iinn rreessppeecctt ooff aarreeaa ttoo bbee hhaannddeedd oovveerr ttoo
B. For redevelopment with tripartite agreement as per 2(b) above:
MMHHAADDAA//MMCCGGMM aanndd sshhaallll bbee eexxcclluussiivvee ooff tthhee FFuunnggiibbllee ccoommppeennssaattoorryy
When there is no developer coming forward for redevelopment of the
BBUUAA iiff aavvaaiilleedd..
building as per above regulation 7(A) and at least 51% or the occupants
PPrroovviiddeedd tthhaatt aatt tthhee ooppttiioonn ooff tthhee OOwwnneerr//DDeevveellooppeerr,, tthhee aarreeaa eeqquuiivvaalleenntt
of at least 5 building adjacent at one location requested
ttoo tthhee mmaarrkkeett vvaalluuee ((aass ppeerr AASSRR ooff tthhaatt yyeeaarr)) ooff aarreeaa aaddmmiissssiibbllee aass ppeerr
MHADA/MCGM to undertake redevelopment by appointing private
tthhee pprreessccrriibbeedd ppeerrcceennttaaggee ooff BBUUAA ttoo MMHHAADDAA//MMCCGGMM ccaann bbee mmaaddee
developer, then in such case MHADA/MCGM may undertake the
aavvaaiillaabbllee wwiitthhiinn tthhee ssaammee oorr aaddjjooiinniinngg mmuunniicciippaall wwaarrdd ooff MMCCGGMM..
redevelopment of cluster of all such buildings belongings to the
330066 DCPR 2034 307 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
bb.. BByy aappppooiinnttiinngg PPrriivvaattee ddeevveellooppeerr bbyy aauutthhoorriittyy wwiitthh ttrriippaarrttyy aaggrreeeemmeenntt
bbeettwweeeenn ddeevveellooppeerr MMHHAADDAA//MMCCGGMM aauutthhoorriittyy aanndd ooccccuuppaannttss 7. FSI Permissible:
aassssoocciiaattiioonn aass ppeerr 77((1133)) ooff tthhiiss rreegguullaattiioonn.. A. For redevelopment undertaken by Private developers as per clause 2(a)
cc.. MMHHAADDAA // MMCCGGMM tthheemmsseellvveess bbyy aappppooiinnttiinngg ddeevveellooppeerr tthhrroouugghh above:
ccoommppeettiittiivvee bbiiddddiinngg aass ppeerr ccllaauussee 77((CC)) ooff tthhiiss rreegguullaattiioonn FSI required for rehabilitation of existing occupants plus incentive
33.. AAllll tthhee eelliiggiibbllee ooccccuuppaannttss iinn bbuuiillddiinngg//ssttrruuccttuurreess sshhaallll bbee cceerrttiiffiieedd bbyy thereon as mentioned below, subject to minimum 3 FSI
rreessppeeccttiivvee aauutthhoorriittiieess oorr MMHHAADDAA // MMCCGGMM..
44.. EEaacchh ooccccuuppaanntt sshhaallll bbee rreehhaabbiilliittaatteedd aanndd ggiivveenn tthhee ccaarrppeett aarreeaa ooccccuuppiieedd Basic Ratio LR/RC Incentive (as % of Admissible rehab Area)
bbyy hhiimm ffoorr rreessiiddeennttiiaall ppuurrppoossee iinn tthhee oolldd bbuuiillddiinngg ssuubbjjeecctt ttoo tthhee For Single Plot For Composite redevelopment of
mmiinniimmuumm ffiixxeedd ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq..fftt..)) aanndd//oorr mmaaxxiimmuumm Two to Five Plots Six or more Plots
ccaarrppeett aarreeaa uuppttoo 112200 ssqq..mm ((11229922 ssqq..fftt..)) IInn ccaassee ooff nnoonn--rreessiiddeennttiiaall Above 6 75 85 90
ooccccuuppiieerr,, tthhee aarreeaa ttoo bbee ggiivveenn iinn tthhee rreeccoonnssttrruucctteedd bbuuiillddiinngg wwiillll bbee Above 4 and upto 6 78 88 95
eeqquuiivvaalleenntt ttoo tthhee aarreeaa ooccccuuppiieedd iinn tthhee oolldd bbuuiillddiinngg.. PPrroovviiddeedd ffuurrtthheerr tthhaatt Upto 4 80 90 100
eeaacchh eelliiggiibbllee rreessiiddeennttiiaall ccuumm ccoommmmeerrcciiaall ooccccuuppaanntt sshhaallll bbee eennttiittlleedd ttoo
aa tteenneemmeenntt ooff mmiinniimmuumm ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq.. mm..))..
Explanation: - RC is rate of construction in respect of RCC
PPrroovviiddeedd ffuurrtthheerr tthhaatt tthhee rreehhaabbiilliittaattiioonn aarreeaa ooff aannyy ooccccuuppaanntt ooff aa
Construction and Land Rate (LR) is the rate of Open Land for FSI 1.
ccoommmmeerrcciiaall eessttaabblliisshhmmeenntt,, wwhhoo iiss aalllloowweedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy
Provided further that in ease there is more than one land rate applicable
MMHHAADDAA//MMCCGGMM ttoo bbee rreehhaabbiilliittaatteedd iinn aa rreessiiddeennttiiaall tteenneemmeenntt iinn lliieeuu
to different parts of the plot under the CDS, a weighted average of all
ooff hhiiss ccoommmmeerrcciiaall eessttaabblliisshhmmeenntt,, sshhaallll aallssoo bbee ggoovveerrnneedd bbyy tthhee
the applicable rates shall be taken for calculating the Average Land Rate
aaffoorreessaaiidd pprroovviissiioonnss aapppplliiccaabbllee ttoo tthhee rreessiiddeennttiiaall ooccccuuppaannttss iinn tthheessee
and the Basic Ratio. Provided further that for calculation of the Basic
RReegguullaattiioonnss..
Ratio, the Land Rate (LR) and the Rate of Construction (RC) shall be
55.. TThhee lliisstt ooff eelliiggiibbllee ooccccuuppaannttss aanndd aarreeaa ooccccuuppiieedd bbyy eeaacchh ooff tthheemm sshhaallll
taken for the year in which the proposal is approved by the Authority
bbee cceerrttiiffiieedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy ii..ee.. MMHHAADDAA oorr MMCCGGMM..
and the said ratio shall remain unchanged even if such Scheme
66.. FFuunnggiibbllee CCoommppeennssaattoorryy AArreeaa aass aapppplliiccaabbllee oonn tthhee aarreeaa ttoo bbee hhaannddeedd
undergoes any revision or modification subsequently during its course
oovveerr ttoo MMHHAADDAA//MMCCGGMM sshhaallll nnoott bbee aalllloowweedd ttoo bbee uuttiilliizzeedd oonn ssaallee
of completion.
ccoommppoonneenntt.. NNoo pprreemmiiuumm sshhaallll bbee cchhaarrggeedd oonn tthhee ffuunnggiibbllee
ccoommppeennssaattoorryy aarreeaa iinn rreessppeecctt ooff aarreeaa ttoo bbee hhaannddeedd oovveerr ttoo
B. For redevelopment with tripartite agreement as per 2(b) above:
MMHHAADDAA//MMCCGGMM aanndd sshhaallll bbee eexxcclluussiivvee ooff tthhee FFuunnggiibbllee ccoommppeennssaattoorryy
When there is no developer coming forward for redevelopment of the
BBUUAA iiff aavvaaiilleedd..
building as per above regulation 7(A) and at least 51% or the occupants
PPrroovviiddeedd tthhaatt aatt tthhee ooppttiioonn ooff tthhee OOwwnneerr//DDeevveellooppeerr,, tthhee aarreeaa eeqquuiivvaalleenntt
of at least 5 building adjacent at one location requested
ttoo tthhee mmaarrkkeett vvaalluuee ((aass ppeerr AASSRR ooff tthhaatt yyeeaarr)) ooff aarreeaa aaddmmiissssiibbllee aass ppeerr
MHADA/MCGM to undertake redevelopment by appointing private
tthhee pprreessccrriibbeedd ppeerrcceennttaaggee ooff BBUUAA ttoo MMHHAADDAA//MMCCGGMM ccaann bbee mmaaddee
developer, then in such case MHADA/MCGM may undertake the
aavvaaiillaabbllee wwiitthhiinn tthhee ssaammee oorr aaddjjooiinniinngg mmuunniicciippaall wwaarrdd ooff MMCCGGMM..
redevelopment of cluster of all such buildings belongings to the
330066 DCPR 2034 307 DCPR 2034


In [32]:
# Sparse search results
display(Markdown("\n**Sparse Search Results:**"))
formatted_results = doc_text_formatting(ef, query, sparse_results)
for result in formatted_results:
    display(Markdown(result))


**Sparse Search Results:**

Original Notifications shall
be referred for perspicuity
redevelopment.
16. Restriction on transfer of tenements shall be governed by provisions
of Maharashtra Rent Control Act, till such time that a Co-op. Housing Society
is formed and thereafter the same shall be governed by the provision of
Maharashtra Co-op. Societies Act. However, tenements constructed for slum
rehabilitation shall not be transferable for a period of 10 years.
17. Corpus Fund: An amount of minimum Rs. 50,000/- per tenement or as
directed by the HPC shall be created by the Promoter/Developer as a
Corpus fund, which will be utilised for maintenance of the
rehabilitation buildings for a period of 10 years.
18. Any ongoing scheme under Regulation<span style='color:red'> 33(7)</span> which fulfills the criteria
under this Regulation can be included<span style='color:red'> in</span> the proposal under Regulation<span style='color:red'>
33</span>(9) for approval or converted into a CDS under this Regulation<span style='color:red'> 33</span>(9).
However, all dilutions of reservations under Regulation<span style='color:red'> 33(7)</span> shall have
to be restored as per this Regulation.
19. Heritage buildings of Grade-I and II as well as authorized and
structurally sound retainable buildings may be included<span style='color:red'> in</span> the CD, but
shall have to be kept as they are, along with land appurtenant, and this
area shall be counted towards the slab of<span style='color:red'> Incentive</span> FSI, but shall not
be considered for FSI under this Regulation. As regards such Heritage
Structures, the Promoter/Developer shall have to contribute Heritage
Cess at 5% of ASR on the basis of BUA of the Heritage structure.
Existing provisions under the DCPR shall apply to Heritage Buildings of
Grade-III and buildings<span style='color:red'> in</span> heritage precincts. However, if the URS
contains Grade I structure, the HPC shall consult the MHCC before
granting approval.
20. If HPC MC/MCGM approves areas for amenities such as Fire
Stations/Hospitals/ Police Stations/Schools, PAP/R&R etc. additional
Area required by the land-owning Authority other than
reservations/designations under the DP, such amenities shall be
handed over to the concerned Authority free of cost. The BUA of such
amenity shall be considered towards rehabilitation FSI, and incentive
FSI as admissible under this Regulation shall be permissible.
21. HPC, headed by Municipal Commissioner and constituted by the Govt.,
shall be competent to approve the CDS with the previous sanction of
the Govt. under this Regulation. On approval by HPC, the proposal shall
be submitted to the Municipal Commissioner, MCGM for approval of
plans. The decision of HPC shall be appealable as if it is an appeal under
section 47 of the MR&TP Act, 1966.
Provided that no CDS shall be sanctioned by the Govt without giving
202 DCPR 2034


Original Notifications shall
be referred for perspicuity
redevelopment.
16. Restriction on transfer of tenements shall be governed by provisions
of Maharashtra Rent Control Act, till such time that a Co-op. Housing Society
is formed and thereafter the same shall be governed by the provision of
Maharashtra Co-op. Societies Act. However, tenements constructed for slum
rehabilitation shall not be transferable for a period of 10 years.
17. Corpus Fund: An amount of minimum Rs. 50,000/- per tenement or as
directed by the HPC shall be created by the Promoter/Developer as a
Corpus fund, which will be utilised for maintenance of the
rehabilitation buildings for a period of 10 years.
18. Any ongoing scheme under Regulation<span style='color:red'> 33(7)</span> which fulfills the criteria
under this Regulation can be included<span style='color:red'> in</span> the proposal under Regulation<span style='color:red'>
33</span>(9) for approval or converted into a CDS under this Regulation<span style='color:red'> 33</span>(9).
However, all dilutions of reservations under Regulation<span style='color:red'> 33(7)</span> shall have
to be restored as per this Regulation.
19. Heritage buildings of Grade-I and II as well as authorized and
structurally sound retainable buildings may be included<span style='color:red'> in</span> the CD, but
shall have to be kept as they are, along with land appurtenant, and this
area shall be counted towards the slab of<span style='color:red'> Incentive</span> FSI, but shall not
be considered for FSI under this Regulation. As regards such Heritage
Structures, the Promoter/Developer shall have to contribute Heritage
Cess at 5% of ASR on the basis of BUA of the Heritage structure.
Existing provisions under the DCPR shall apply to Heritage Buildings of
Grade-III and buildings<span style='color:red'> in</span> heritage precincts. However, if the URS
contains Grade I structure, the HPC shall consult the MHCC before
granting approval.
20. If HPC MC/MCGM approves areas for amenities such as Fire
Stations/Hospitals/ Police Stations/Schools, PAP/R&R etc. additional
Area required by the land-owning Authority other than
reservations/designations under the DP, such amenities shall be
handed over to the concerned Authority free of cost. The BUA of such
amenity shall be considered towards rehabilitation FSI, and incentive
FSI as admissible under this Regulation shall be permissible.
21. HPC, headed by Municipal Commissioner and constituted by the Govt.,
shall be competent to approve the CDS with the previous sanction of
the Govt. under this Regulation. On approval by HPC, the proposal shall
be submitted to the Municipal Commissioner, MCGM for approval of
plans. The decision of HPC shall be appealable as if it is an appeal under
section 47 of the MR&TP Act, 1966.
Provided that no CDS shall be sanctioned by the Govt without giving
202 DCPR 2034


Original Notifications shall
be referred for perspicuity
redevelopment.
16. Restriction on transfer of tenements shall be governed by provisions
of Maharashtra Rent Control Act, till such time that a Co-op. Housing Society
is formed and thereafter the same shall be governed by the provision of
Maharashtra Co-op. Societies Act. However, tenements constructed for slum
rehabilitation shall not be transferable for a period of 10 years.
17. Corpus Fund: An amount of minimum Rs. 50,000/- per tenement or as
directed by the HPC shall be created by the Promoter/Developer as a
Corpus fund, which will be utilised for maintenance of the
rehabilitation buildings for a period of 10 years.
18. Any ongoing scheme under Regulation<span style='color:red'> 33(7)</span> which fulfills the criteria
under this Regulation can be included<span style='color:red'> in</span> the proposal under Regulation<span style='color:red'>
33</span>(9) for approval or converted into a CDS under this Regulation<span style='color:red'> 33</span>(9).
However, all dilutions of reservations under Regulation<span style='color:red'> 33(7)</span> shall have
to be restored as per this Regulation.
19. Heritage buildings of Grade-I and II as well as authorized and
structurally sound retainable buildings may be included<span style='color:red'> in</span> the CD, but
shall have to be kept as they are, along with land appurtenant, and this
area shall be counted towards the slab of<span style='color:red'> Incentive</span> FSI, but shall not
be considered for FSI under this Regulation. As regards such Heritage
Structures, the Promoter/Developer shall have to contribute Heritage
Cess at 5% of ASR on the basis of BUA of the Heritage structure.
Existing provisions under the DCPR shall apply to Heritage Buildings of
Grade-III and buildings<span style='color:red'> in</span> heritage precincts. However, if the URS
contains Grade I structure, the HPC shall consult the MHCC before
granting approval.
20. If HPC MC/MCGM approves areas for amenities such as Fire
Stations/Hospitals/ Police Stations/Schools, PAP/R&R etc. additional
Area required by the land-owning Authority other than
reservations/designations under the DP, such amenities shall be
handed over to the concerned Authority free of cost. The BUA of such
amenity shall be considered towards rehabilitation FSI, and incentive
FSI as admissible under this Regulation shall be permissible.
21. HPC, headed by Municipal Commissioner and constituted by the Govt.,
shall be competent to approve the CDS with the previous sanction of
the Govt. under this Regulation. On approval by HPC, the proposal shall
be submitted to the Municipal Commissioner, MCGM for approval of
plans. The decision of HPC shall be appealable as if it is an appeal under
section 47 of the MR&TP Act, 1966.
Provided that no CDS shall be sanctioned by the Govt without giving
202 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
mmeeaannss tthhee nneett uussaabbllee fflloooorr aarreeaa wwiitthhiinn aa tteenneemme<span style='color:red'>en</span>ntt eexxcclluuddiinngg tthhaatt
ccoovveerreedd bbyy tthhee wwaallllss oorr aannyy ootthheerr aarreeaass ssppeecciiffiiccaallllyy eexxeemmpptteedd ffrroomm
fflloooorr ssppaaccee iinnddeexx ccoommppuuttaattiioonn aass ppeerr tthheenn//pprreevvaaiilliinngg RReegguullaattiioonn bbuutt Basic Ratio<span style='color:red'> Incentive</span> (As % of Admissible Rehabilitation Area)
(LR/RC)
iinncclluuddiinngg tthhee aarreeaass ooff bbaallccoonnyy iiff aalllloowweedd ffrreeee ooff FFSSII aass ppeerr tthheenn
RReegguullaattiioonn For 0.4 ha up More than 1 More than 5 For more
to 1 ha. ha up to 5 ha ha up to 10 ha. than 10 ha
Above 6.00 85% 90% 95% 100%
Above 4.00 95% 100% 105% 110%
66.. TToottaall PPeerrmmiissssiibbllee FFSSII ffoorr CCDDSS::
And upto 6.00
aa)) TThhee ttoottaall ppeerrmmiissssiibbllee FFSSII ffoorr aann CCDDSS sshhaallll bbee 44..0000 oonn ggrroossss pplloott aarreeaa,, bbuutt
Above 2.00 and 105% 110% 115% 120%
eexxcclluuddiinngg tthhee rreesseerrvvaattiioonnss// eexxiissttiinngg aamme<span style='color:red'>en</span>niittyy,, rrooaadd sseett bbaacckk,, aarreeaa uunnddeerr
upto 4.00
eexxiissttiinngg MMuunniicciippaall RRooaaddss bbuutt iinncclluuddiinngg tthhee BBUUAA uunnddeerr
Upto 2.00 115% 120% 125% 130%
rreesseerrvvaattiioonn//eexxiissttiinngg aamme<span style='color:red'>en</span>niittyy,, rrooaadd sseett bbaacckk oorr ssuumm ttoottaall ooff tthhee
RReehhaabbiilliittaattiioonn FFSSII ++ IInncce<span style='color:red'>en</span>nttiivvee FFSSII,, wwhhiicchheevveerr iiss mmoorree..
Explanation: -
PPrroovviiddeedd tthhaatt tthhee aaffoorreessaaiidd FFSSII sshhaallll bbee eexxcclluussiivvee ooff tthhee FFuunnggiibbllee
CCoommppe<span style='color:red'>en</span>nssaattoorryy aarreeaa aaddmmiissssiibbllee uunnddeerr tthhee pprroovviissiioonn ooff DDCCRR<span style='color:red'> 33</span>11<span style='color:red'>((</span>33))..
(i) *RC is rate of construction<span style='color:red'> in</span> respect of RCC Construction and Land
bb)) TThhee iinncce<span style='color:red'>en</span>nttiivvee FFSSII aaddmmiissssiibbllee aaggaaiinnsstt tthhee FFSSII rreeqquuiirreedd ffoorr rreehhaabbiilliittaattiioonn Rate (LR) is the rate of Open Land for FSI 1.
sshhaallll bbee bbaasseedd oonn tthhee rraattiioo ((hheerreeiinnaafftteerr rreeffeerrrreedd ttoo aass BBaassiicc RRaattiioo)) ooff Provided further that<span style='color:red'> in</span> case there is more than one land rate
LLaanndd RRaattee ((LLRR)),, iinn RRss//ssqq.. mm..,, ooff tthhee llaannddss iinncclluuddeedd iinn tthhee UURRCC;; aass ppeerr tthhee applicable to different parts of the plot under the CDS, a weighted
AASSRR aanndd RRaattee ooff CCoonnssttrruuccttiioonn ((RRCC))** iinn RRss//ssqq.. mm,, aapppplliiccaabbllee ttoo tthhee aarreeaa average of all the applicable rates shall be taken for calculating the
aass ppeerr tthhee AASSRR aanndd sshhaallll bbee ggiivve<span style='color:red'>en</span>n aass ppeerr tthhee TTaabbllee--BB bbeellooww:: Average Land Rate and the Basic Ratio.
Provided further that for calculation of the Basic Ratio, the Land Rate
TTaabbllee--BB (LR) and the Rate of Construction (RC) shall be taken for the year<span style='color:red'> in</span>
which the Cluster Development Project is approved and LOI is issued
by the Authority competent to approve it and the said ratio shall
BBaassiicc IInncce<span style='color:red'>en</span>nttiivvee ((AAss %% ooff AAddmmiissssiibbllee RReehhaabbiilliittaattiioonn AArreeaa))
remain unchanged even if such Scheme undergoes any revision or
RRaattiioo
modification subsequently during its course of completion.
((LLRR//RRCC))**
FFoorr 00..44 hhaa uupp MMoorree tthhaann MMoorree tthhaann 55 FFoorr Provided further that if any new area is allowed to be added to or
ttoo 11 hhaa 11 hhaa uupp ttoo 55 hhaa uupp ttoo 1100 mmoorr ee deleted from the CDS after such Scheme has been approved and if
hhaa hhaa.. tthhaann there is change<span style='color:red'> in</span> the slab prescribed above, the incentive FSI for the
1100hhaa
total area of the revised Scheme shall be determined as per the new
AAbboovvee 5555%% 6600%% 6655%% 7700%%
slab. Provided further that any new area being added to a CDS shall
66..0000
not be less than 75% of the minimum area required for CDS.
AAbboovvee 6655%% 7700%% 7755%% 8800%% Provided further that augmentation of area of CD shall not be allowed
44..0000 aanndd after further CC has been issued<span style='color:red'> in</span> respect of more than 75% of the
uuppttoo 66..0000
total permissible BUA sanctioned under the original Scheme and there
AAbboovvee 7755%% 8800%% 8855%% 9900%%
shall be no revision of individual areas as a result of such amalgamation
22..0000 aanndd
of area. However, deletion of area from a sanctioned scheme will be
uuppttoo 44..0000
permissible, provided the construction of rehabilitation component
UUppttoo 8855%% 9900%% 9955%% 110000%%
22..0000 has not commenced and such deletion does not break the contiguity
of the area under CDS.
119944 DCPR 2034 195 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
mmeeaannss tthhee nneett uussaabbllee fflloooorr aarreeaa wwiitthhiinn aa tteenneemme<span style='color:red'>en</span>ntt eexxcclluuddiinngg tthhaatt
ccoovveerreedd bbyy tthhee wwaallllss oorr aannyy ootthheerr aarreeaass ssppeecciiffiiccaallllyy eexxeemmpptteedd ffrroomm
fflloooorr ssppaaccee iinnddeexx ccoommppuuttaattiioonn aass ppeerr tthheenn//pprreevvaaiilliinngg RReegguullaattiioonn bbuutt Basic Ratio<span style='color:red'> Incentive</span> (As % of Admissible Rehabilitation Area)
(LR/RC)
iinncclluuddiinngg tthhee aarreeaass ooff bbaallccoonnyy iiff aalllloowweedd ffrreeee ooff FFSSII aass ppeerr tthheenn
RReegguullaattiioonn For 0.4 ha up More than 1 More than 5 For more
to 1 ha. ha up to 5 ha ha up to 10 ha. than 10 ha
Above 6.00 85% 90% 95% 100%
Above 4.00 95% 100% 105% 110%
66.. TToottaall PPeerrmmiissssiibbllee FFSSII ffoorr CCDDSS::
And upto 6.00
aa)) TThhee ttoottaall ppeerrmmiissssiibbllee FFSSII ffoorr aann CCDDSS sshhaallll bbee 44..0000 oonn ggrroossss pplloott aarreeaa,, bbuutt
Above 2.00 and 105% 110% 115% 120%
eexxcclluuddiinngg tthhee rreesseerrvvaattiioonnss// eexxiissttiinngg aamme<span style='color:red'>en</span>niittyy,, rrooaadd sseett bbaacckk,, aarreeaa uunnddeerr
upto 4.00
eexxiissttiinngg MMuunniicciippaall RRooaaddss bbuutt iinncclluuddiinngg tthhee BBUUAA uunnddeerr
Upto 2.00 115% 120% 125% 130%
rreesseerrvvaattiioonn//eexxiissttiinngg aamme<span style='color:red'>en</span>niittyy,, rrooaadd sseett bbaacckk oorr ssuumm ttoottaall ooff tthhee
RReehhaabbiilliittaattiioonn FFSSII ++ IInncce<span style='color:red'>en</span>nttiivvee FFSSII,, wwhhiicchheevveerr iiss mmoorree..
Explanation: -
PPrroovviiddeedd tthhaatt tthhee aaffoorreessaaiidd FFSSII sshhaallll bbee eexxcclluussiivvee ooff tthhee FFuunnggiibbllee
CCoommppe<span style='color:red'>en</span>nssaattoorryy aarreeaa aaddmmiissssiibbllee uunnddeerr tthhee pprroovviissiioonn ooff DDCCRR<span style='color:red'> 33</span>11<span style='color:red'>((</span>33))..
(i) *RC is rate of construction<span style='color:red'> in</span> respect of RCC Construction and Land
bb)) TThhee iinncce<span style='color:red'>en</span>nttiivvee FFSSII aaddmmiissssiibbllee aaggaaiinnsstt tthhee FFSSII rreeqquuiirreedd ffoorr rreehhaabbiilliittaattiioonn Rate (LR) is the rate of Open Land for FSI 1.
sshhaallll bbee bbaasseedd oonn tthhee rraattiioo ((hheerreeiinnaafftteerr rreeffeerrrreedd ttoo aass BBaassiicc RRaattiioo)) ooff Provided further that<span style='color:red'> in</span> case there is more than one land rate
LLaanndd RRaattee ((LLRR)),, iinn RRss//ssqq.. mm..,, ooff tthhee llaannddss iinncclluuddeedd iinn tthhee UURRCC;; aass ppeerr tthhee applicable to different parts of the plot under the CDS, a weighted
AASSRR aanndd RRaattee ooff CCoonnssttrruuccttiioonn ((RRCC))** iinn RRss//ssqq.. mm,, aapppplliiccaabbllee ttoo tthhee aarreeaa average of all the applicable rates shall be taken for calculating the
aass ppeerr tthhee AASSRR aanndd sshhaallll bbee ggiivve<span style='color:red'>en</span>n aass ppeerr tthhee TTaabbllee--BB bbeellooww:: Average Land Rate and the Basic Ratio.
Provided further that for calculation of the Basic Ratio, the Land Rate
TTaabbllee--BB (LR) and the Rate of Construction (RC) shall be taken for the year<span style='color:red'> in</span>
which the Cluster Development Project is approved and LOI is issued
by the Authority competent to approve it and the said ratio shall
BBaassiicc IInncce<span style='color:red'>en</span>nttiivvee ((AAss %% ooff AAddmmiissssiibbllee RReehhaabbiilliittaattiioonn AArreeaa))
remain unchanged even if such Scheme undergoes any revision or
RRaattiioo
modification subsequently during its course of completion.
((LLRR//RRCC))**
FFoorr 00..44 hhaa uupp MMoorree tthhaann MMoorree tthhaann 55 FFoorr Provided further that if any new area is allowed to be added to or
ttoo 11 hhaa 11 hhaa uupp ttoo 55 hhaa uupp ttoo 1100 mmoorr ee deleted from the CDS after such Scheme has been approved and if
hhaa hhaa.. tthhaann there is change<span style='color:red'> in</span> the slab prescribed above, the incentive FSI for the
1100hhaa
total area of the revised Scheme shall be determined as per the new
AAbboovvee 5555%% 6600%% 6655%% 7700%%
slab. Provided further that any new area being added to a CDS shall
66..0000
not be less than 75% of the minimum area required for CDS.
AAbboovvee 6655%% 7700%% 7755%% 8800%% Provided further that augmentation of area of CD shall not be allowed
44..0000 aanndd after further CC has been issued<span style='color:red'> in</span> respect of more than 75% of the
uuppttoo 66..0000
total permissible BUA sanctioned under the original Scheme and there
AAbboovvee 7755%% 8800%% 8855%% 9900%%
shall be no revision of individual areas as a result of such amalgamation
22..0000 aanndd
of area. However, deletion of area from a sanctioned scheme will be
uuppttoo 44..0000
permissible, provided the construction of rehabilitation component
UUppttoo 8855%% 9900%% 9955%% 110000%%
22..0000 has not commenced and such deletion does not break the contiguity
of the area under CDS.
119944 DCPR 2034 195 DCPR 2034


In [33]:
# Hybrid search results
display(Markdown("\n**Hybrid Search Results:**"))
formatted_results = doc_text_formatting(ef, query, hybrid_results)
for result in formatted_results:
    display(Markdown(result))


**Hybrid Search Results:**

Original Notifications shall
be referred for perspicuity<span style='color:red'>
33(7)(</span>B) Additional FSI for Redevelopment of existing
residential housing societies excluding buildings covered
under regulation<span style='color:red'> 33(7)</span> and<span style='color:red'> 33(7)(</span>A):
1) In case of redevelopment of existing residential housing societies
excluding buildings covered under regulation<span style='color:red'> 33(7)</span> and<span style='color:red'> 33(7)(</span>A)
proposed by Housing societies/landlords or through their proponents
where existing members are proposed to be re- accommodated on the
same plot, incentive additional BUA to the extent of 15% of existing
BUA or 10 sq. m per tenement whichever is more shall be permissible
without premium.
Provided further that if the existing authorized BUA and incentive
thereon as stated above is less than the permissible FSI as per
regulation 30<span style='color:red'>(</span>A)(1), then society may avail ‘Additional FSI on payment
of premium/TDR’ up to limit of permissible FSI.
2) If staircase, lift & lift lobby areas are claimed free of FSI by charging
premium as per then prevailing Regulation, then such areas to that
extent only will be granted free of FSI without charging premium. If
staircase, lift & lift lobby areas are counted<span style='color:red'> in</span> FSI<span style='color:red'> in</span> earlier
development, then incentive additional FSI as stated<span style='color:red'> in</span> Sr. No. 1 shall
also be given on such area & such areas may be availed free of FSI by
charging premium as per these Regulations.
3) This Regulation shall be applicable only when existing members of the
societies are proposed to be re-accommodated.
4) This regulation will be applicable for redevelopment of existing
authorized buildings which are of thirty years of age or more.
5) This regulation shall not be applicable<span style='color:red'> in</span> respect of redevelopment
proposal to be/being processed under Regulation No.<span style='color:red'> 33</span>(5),<span style='color:red'> 33(7)</span>,<span style='color:red'>
33(</span>8),<span style='color:red'> 33</span>(9),<span style='color:red'> 33</span>(9)<span style='color:red'>(</span>A),<span style='color:red'> 33</span>(9)<span style='color:red'>(</span>B),<span style='color:red'> 33</span>(10),<span style='color:red'> 33</span>(10) (A),<span style='color:red'> 33(</span>20) (A),<span style='color:red'> 33(</span>21).
Explanation: -Age of a building shall be as on the 1st of January of the
year<span style='color:red'> in</span> which a complete redevelopment proposal is submitted to the
Commissioner and shall be calculated from the date of Occupation
Certificate or alternately, from the first date of assessment as per the
property tax record<span style='color:red'> in</span> respect of such building or building on which
balance/admissible FSI has been consumed by way of vertical
extension as per then prevailing Regulation, available with the MCGM.
6) This incentive additional BUA shall be independent of additional BUA
as permissible under Regulation No. 14<span style='color:red'>(</span>A), 15, 16 and 17, if any.
7) Fungible compensatory area admissible under Regulation No. 31(3) on
the existing authorised BUA shall be without charging premium and
over the incentive additional BUA by charging of premium.
8) If tenanted building/s and building/s of co-operative housing
society/non- tenanted building/s co-exist on the plot under
development, then proportionate land component as per the existing
authorised BUA of existing tenanted building on the plot shall be
180 DCPR 2034


Original Notifications shall
be referred for perspicuity<span style='color:red'>
33(7)(</span>B) Additional FSI for Redevelopment of existing
residential housing societies excluding buildings covered
under regulation<span style='color:red'> 33(7)</span> and<span style='color:red'> 33(7)(</span>A):
1) In case of redevelopment of existing residential housing societies
excluding buildings covered under regulation<span style='color:red'> 33(7)</span> and<span style='color:red'> 33(7)(</span>A)
proposed by Housing societies/landlords or through their proponents
where existing members are proposed to be re- accommodated on the
same plot, incentive additional BUA to the extent of 15% of existing
BUA or 10 sq. m per tenement whichever is more shall be permissible
without premium.
Provided further that if the existing authorized BUA and incentive
thereon as stated above is less than the permissible FSI as per
regulation 30<span style='color:red'>(</span>A)(1), then society may avail ‘Additional FSI on payment
of premium/TDR’ up to limit of permissible FSI.
2) If staircase, lift & lift lobby areas are claimed free of FSI by charging
premium as per then prevailing Regulation, then such areas to that
extent only will be granted free of FSI without charging premium. If
staircase, lift & lift lobby areas are counted<span style='color:red'> in</span> FSI<span style='color:red'> in</span> earlier
development, then incentive additional FSI as stated<span style='color:red'> in</span> Sr. No. 1 shall
also be given on such area & such areas may be availed free of FSI by
charging premium as per these Regulations.
3) This Regulation shall be applicable only when existing members of the
societies are proposed to be re-accommodated.
4) This regulation will be applicable for redevelopment of existing
authorized buildings which are of thirty years of age or more.
5) This regulation shall not be applicable<span style='color:red'> in</span> respect of redevelopment
proposal to be/being processed under Regulation No.<span style='color:red'> 33</span>(5),<span style='color:red'> 33(7)</span>,<span style='color:red'>
33(</span>8),<span style='color:red'> 33</span>(9),<span style='color:red'> 33</span>(9)<span style='color:red'>(</span>A),<span style='color:red'> 33</span>(9)<span style='color:red'>(</span>B),<span style='color:red'> 33</span>(10),<span style='color:red'> 33</span>(10) (A),<span style='color:red'> 33(</span>20) (A),<span style='color:red'> 33(</span>21).
Explanation: -Age of a building shall be as on the 1st of January of the
year<span style='color:red'> in</span> which a complete redevelopment proposal is submitted to the
Commissioner and shall be calculated from the date of Occupation
Certificate or alternately, from the first date of assessment as per the
property tax record<span style='color:red'> in</span> respect of such building or building on which
balance/admissible FSI has been consumed by way of vertical
extension as per then prevailing Regulation, available with the MCGM.
6) This incentive additional BUA shall be independent of additional BUA
as permissible under Regulation No. 14<span style='color:red'>(</span>A), 15, 16 and 17, if any.
7) Fungible compensatory area admissible under Regulation No. 31(3) on
the existing authorised BUA shall be without charging premium and
over the incentive additional BUA by charging of premium.
8) If tenanted building/s and building/s of co-operative housing
society/non- tenanted building/s co-exist on the plot under
development, then proportionate land component as per the existing
authorised BUA of existing tenanted building on the plot shall be
180 DCPR 2034


Original Notifications shall
be referred for perspicuity<span style='color:red'>
33(7)(</span>B) Additional FSI for Redevelopment of existing
residential housing societies excluding buildings covered
under regulation<span style='color:red'> 33(7)</span> and<span style='color:red'> 33(7)(</span>A):
1) In case of redevelopment of existing residential housing societies
excluding buildings covered under regulation<span style='color:red'> 33(7)</span> and<span style='color:red'> 33(7)(</span>A)
proposed by Housing societies/landlords or through their proponents
where existing members are proposed to be re- accommodated on the
same plot, incentive additional BUA to the extent of 15% of existing
BUA or 10 sq. m per tenement whichever is more shall be permissible
without premium.
Provided further that if the existing authorized BUA and incentive
thereon as stated above is less than the permissible FSI as per
regulation 30<span style='color:red'>(</span>A)(1), then society may avail ‘Additional FSI on payment
of premium/TDR’ up to limit of permissible FSI.
2) If staircase, lift & lift lobby areas are claimed free of FSI by charging
premium as per then prevailing Regulation, then such areas to that
extent only will be granted free of FSI without charging premium. If
staircase, lift & lift lobby areas are counted<span style='color:red'> in</span> FSI<span style='color:red'> in</span> earlier
development, then incentive additional FSI as stated<span style='color:red'> in</span> Sr. No. 1 shall
also be given on such area & such areas may be availed free of FSI by
charging premium as per these Regulations.
3) This Regulation shall be applicable only when existing members of the
societies are proposed to be re-accommodated.
4) This regulation will be applicable for redevelopment of existing
authorized buildings which are of thirty years of age or more.
5) This regulation shall not be applicable<span style='color:red'> in</span> respect of redevelopment
proposal to be/being processed under Regulation No.<span style='color:red'> 33</span>(5),<span style='color:red'> 33(7)</span>,<span style='color:red'>
33(</span>8),<span style='color:red'> 33</span>(9),<span style='color:red'> 33</span>(9)<span style='color:red'>(</span>A),<span style='color:red'> 33</span>(9)<span style='color:red'>(</span>B),<span style='color:red'> 33</span>(10),<span style='color:red'> 33</span>(10) (A),<span style='color:red'> 33(</span>20) (A),<span style='color:red'> 33(</span>21).
Explanation: -Age of a building shall be as on the 1st of January of the
year<span style='color:red'> in</span> which a complete redevelopment proposal is submitted to the
Commissioner and shall be calculated from the date of Occupation
Certificate or alternately, from the first date of assessment as per the
property tax record<span style='color:red'> in</span> respect of such building or building on which
balance/admissible FSI has been consumed by way of vertical
extension as per then prevailing Regulation, available with the MCGM.
6) This incentive additional BUA shall be independent of additional BUA
as permissible under Regulation No. 14<span style='color:red'>(</span>A), 15, 16 and 17, if any.
7) Fungible compensatory area admissible under Regulation No. 31(3) on
the existing authorised BUA shall be without charging premium and
over the incentive additional BUA by charging of premium.
8) If tenanted building/s and building/s of co-operative housing
society/non- tenanted building/s co-exist on the plot under
development, then proportionate land component as per the existing
authorised BUA of existing tenanted building on the plot shall be
180 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
bb.. BByy aappppooiinnttiinngg PPrriivvaattee ddeevveellooppeerr bbyy aauutthhoorriittyy wwiitthh ttrriippaarrttyy aaggrreeeemme<span style='color:red'>en</span>ntt
bbeettwweeeenn ddeevveellooppeerr MMHHAADDAA//MMCCGGMM aauutthhoorriittyy aanndd ooccccuuppaannttss 7. FSI Permissible:
aassssoocciiaattiioonn aass ppeerr 77<span style='color:red'>((</span>1133)) ooff tthhiiss rreegguullaattiioonn.. A. For redevelopment undertak<span style='color:red'>en</span> by Private developers as per clause 2<span style='color:red'>(</span>a)
cc.. MMHHAADDAA // MMCCGGMM tthheemmsseellvveess bbyy aappppooiinnttiinngg ddeevveellooppeerr tthhrroouugghh above:
ccoommppeettiittiivvee bbiiddddiinngg aass ppeerr ccllaauussee 77<span style='color:red'>((</span>CC)) ooff tthhiiss rreegguullaattiioonn FSI required for rehabilitation of existing occupants plus incentive
33.. AAllll tthhee eelliiggiibbllee ooccccuuppaannttss iinn bbuuiillddiinngg//ssttrruuccttuurreess sshhaallll bbee cceerrttiiffiieedd bbyy thereon as mentioned below, subject to minimum 3 FSI
rreessppeeccttiivvee aauutthhoorriittiieess oorr MMHHAADDAA // MMCCGGMM..
44.. EEaacchh ooccccuuppaanntt sshhaallll bbee rreehhaabbiilliittaatteedd aanndd ggiivve<span style='color:red'>en</span>n tthhee ccaarrppeett aarreeaa ooccccuuppiieedd Basic Ratio LR/RC<span style='color:red'> Incentive</span> (as % of Admissible rehab Area)
bbyy hhiimm ffoorr rreessiidde<span style='color:red'>en</span>nttiiaall ppuurrppoossee iinn tthhee oolldd bbuuiillddiinngg ssuubbjjeecctt ttoo tthhee For Single Plot For Composite redevelopment of
mmiinniimmuumm ffiixxeedd ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq..fftt..)) aanndd//oorr mmaaxxiimmuumm Two to Five Plots Six or more Plots
ccaarrppeett aarreeaa uuppttoo 112200 ssqq..mm ((11229922 ssqq..fftt..)) IInn ccaassee ooff nnoonn--rreessiidde<span style='color:red'>en</span>nttiiaall Above 6 75 85 90
ooccccuuppiieerr,, tthhee aarreeaa ttoo bbee ggiivve<span style='color:red'>en</span>n iinn tthhee rreeccoonnssttrruucctteedd bbuuiillddiinngg wwiillll bbee Above 4 and upto 6 78 88 95
eeqquuiivvaalleenntt ttoo tthhee aarreeaa ooccccuuppiieedd iinn tthhee oolldd bbuuiillddiinngg.. PPrroovviiddeedd ffuurrtthheerr tthhaatt Upto 4 80 90 100
eeaacchh eelliiggiibbllee rreessiidde<span style='color:red'>en</span>nttiiaall ccuumm ccoommmmeerrcciiaall ooccccuuppaanntt sshhaallll bbee eennttiittlleedd ttoo
aa tteenneemme<span style='color:red'>en</span>ntt ooff mmiinniimmuumm ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq.. mm..))..
Explanation: - RC is rate of construction<span style='color:red'> in</span> respect of RCC
PPrroovviiddeedd ffuurrtthheerr tthhaatt tthhee rreehhaabbiilliittaattiioonn aarreeaa ooff aannyy ooccccuuppaanntt ooff aa
Construction and Land Rate (LR) is the rate of Open Land for FSI 1.
ccoommmmeerrcciiaall eessttaabblliisshhmme<span style='color:red'>en</span>ntt,, wwhhoo iiss aalllloowweedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy
Provided further that<span style='color:red'> in</span> ease there is more than one land rate applicable
MMHHAADDAA//MMCCGGMM ttoo bbee rreehhaabbiilliittaatteedd iinn aa rreessiidde<span style='color:red'>en</span>nttiiaall tteenneemme<span style='color:red'>en</span>ntt iinn lliieeuu
to different parts of the plot under the CDS, a weighted average of all
ooff hhiiss ccoommmmeerrcciiaall eessttaabblliisshhmme<span style='color:red'>en</span>ntt,, sshhaallll aallssoo bbee ggoovveerrnneedd bbyy tthhee
the applicable rates shall be taken for calculating the Average Land Rate
aaffoorreessaaiidd pprroovviissiioonnss aapppplliiccaabbllee ttoo tthhee rreessiidde<span style='color:red'>en</span>nttiiaall ooccccuuppaannttss iinn tthheessee
and the Basic Ratio. Provided further that for calculation of the Basic
RReegguullaattiioonnss..
Ratio, the Land Rate (LR) and the Rate of Construction (RC) shall be
55.. TThhee lliisstt ooff eelliiggiibbllee ooccccuuppaannttss aanndd aarreeaa ooccccuuppiieedd bbyy eeaacchh ooff tthheemm sshhaallll
taken for the year<span style='color:red'> in</span> which the proposal is approved by the Authority
bbee cceerrttiiffiieedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy ii..ee.. MMHHAADDAA oorr MMCCGGMM..
and the said ratio shall remain unchanged even if such Scheme
66.. FFuunnggiibbllee CCoommppe<span style='color:red'>en</span>nssaattoorryy AArreeaa aass aapppplliiccaabbllee oonn tthhee aarreeaa ttoo bbee hhaannddeedd
undergoes any revision or modification subsequently during its course
oovveerr ttoo MMHHAADDAA//MMCCGGMM sshhaallll nnoott bbee aalllloowweedd ttoo bbee uuttiilliizzeedd oonn ssaallee
of completion.
ccoommppoonne<span style='color:red'>en</span>ntt.. NNoo pprreemmiiuumm sshhaallll bbee cchhaarrggeedd oonn tthhee ffuunnggiibbllee
ccoommppe<span style='color:red'>en</span>nssaattoorryy aarreeaa iinn rreessppeecctt ooff aarreeaa ttoo bbee hhaannddeedd oovveerr ttoo
B. For redevelopment with tripartite agreement as per 2<span style='color:red'>(</span>b) above:
MMHHAADDAA//MMCCGGMM aanndd sshhaallll bbee eexxcclluussiivvee ooff tthhee FFuunnggiibbllee ccoommppe<span style='color:red'>en</span>nssaattoorryy
When there is no developer coming forward for redevelopment of the
BBUUAA iiff aavvaaiilleedd..
building as per above regulation 7<span style='color:red'>(</span>A) and at least 51% or the occupants
PPrroovviiddeedd tthhaatt aatt tthhee ooppttiioonn ooff tthhee OOwwnneerr//DDeevveellooppeerr,, tthhee aarreeaa eeqquuiivvaalleenntt
of at least 5 building adjacent at one location requested
ttoo tthhee mmaarrkkeett vvaalluuee ((aass ppeerr AASSRR ooff tthhaatt yyeeaarr)) ooff aarreeaa aaddmmiissssiibbllee aass ppeerr
MHADA/MCGM to undertake redevelopment by appointing private
tthhee pprreessccrriibbeedd ppeerrcce<span style='color:red'>en</span>nttaaggee ooff BBUUAA ttoo MMHHAADDAA//MMCCGGMM ccaann bbee mmaaddee
developer, then<span style='color:red'> in</span> such case MHADA/MCGM may undertake the
aavvaaiillaabbllee wwiitthhiinn tthhee ssaammee oorr aaddjjooiinniinngg mmuunniicciippaall wwaarrdd ooff MMCCGGMM..
redevelopment of cluster of all such buildings belongings to the
330066 DCPR 2034 307 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
bb.. BByy aappppooiinnttiinngg PPrriivvaattee ddeevveellooppeerr bbyy aauutthhoorriittyy wwiitthh ttrriippaarrttyy aaggrreeeemme<span style='color:red'>en</span>ntt
bbeettwweeeenn ddeevveellooppeerr MMHHAADDAA//MMCCGGMM aauutthhoorriittyy aanndd ooccccuuppaannttss 7. FSI Permissible:
aassssoocciiaattiioonn aass ppeerr 77<span style='color:red'>((</span>1133)) ooff tthhiiss rreegguullaattiioonn.. A. For redevelopment undertak<span style='color:red'>en</span> by Private developers as per clause 2<span style='color:red'>(</span>a)
cc.. MMHHAADDAA // MMCCGGMM tthheemmsseellvveess bbyy aappppooiinnttiinngg ddeevveellooppeerr tthhrroouugghh above:
ccoommppeettiittiivvee bbiiddddiinngg aass ppeerr ccllaauussee 77<span style='color:red'>((</span>CC)) ooff tthhiiss rreegguullaattiioonn FSI required for rehabilitation of existing occupants plus incentive
33.. AAllll tthhee eelliiggiibbllee ooccccuuppaannttss iinn bbuuiillddiinngg//ssttrruuccttuurreess sshhaallll bbee cceerrttiiffiieedd bbyy thereon as mentioned below, subject to minimum 3 FSI
rreessppeeccttiivvee aauutthhoorriittiieess oorr MMHHAADDAA // MMCCGGMM..
44.. EEaacchh ooccccuuppaanntt sshhaallll bbee rreehhaabbiilliittaatteedd aanndd ggiivve<span style='color:red'>en</span>n tthhee ccaarrppeett aarreeaa ooccccuuppiieedd Basic Ratio LR/RC<span style='color:red'> Incentive</span> (as % of Admissible rehab Area)
bbyy hhiimm ffoorr rreessiidde<span style='color:red'>en</span>nttiiaall ppuurrppoossee iinn tthhee oolldd bbuuiillddiinngg ssuubbjjeecctt ttoo tthhee For Single Plot For Composite redevelopment of
mmiinniimmuumm ffiixxeedd ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq..fftt..)) aanndd//oorr mmaaxxiimmuumm Two to Five Plots Six or more Plots
ccaarrppeett aarreeaa uuppttoo 112200 ssqq..mm ((11229922 ssqq..fftt..)) IInn ccaassee ooff nnoonn--rreessiidde<span style='color:red'>en</span>nttiiaall Above 6 75 85 90
ooccccuuppiieerr,, tthhee aarreeaa ttoo bbee ggiivve<span style='color:red'>en</span>n iinn tthhee rreeccoonnssttrruucctteedd bbuuiillddiinngg wwiillll bbee Above 4 and upto 6 78 88 95
eeqquuiivvaalleenntt ttoo tthhee aarreeaa ooccccuuppiieedd iinn tthhee oolldd bbuuiillddiinngg.. PPrroovviiddeedd ffuurrtthheerr tthhaatt Upto 4 80 90 100
eeaacchh eelliiggiibbllee rreessiidde<span style='color:red'>en</span>nttiiaall ccuumm ccoommmmeerrcciiaall ooccccuuppaanntt sshhaallll bbee eennttiittlleedd ttoo
aa tteenneemme<span style='color:red'>en</span>ntt ooff mmiinniimmuumm ccaarrppeett aarreeaa ooff 2277..8888 ssqq.. mm ((330000 ssqq.. mm..))..
Explanation: - RC is rate of construction<span style='color:red'> in</span> respect of RCC
PPrroovviiddeedd ffuurrtthheerr tthhaatt tthhee rreehhaabbiilliittaattiioonn aarreeaa ooff aannyy ooccccuuppaanntt ooff aa
Construction and Land Rate (LR) is the rate of Open Land for FSI 1.
ccoommmmeerrcciiaall eessttaabblliisshhmme<span style='color:red'>en</span>ntt,, wwhhoo iiss aalllloowweedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy
Provided further that<span style='color:red'> in</span> ease there is more than one land rate applicable
MMHHAADDAA//MMCCGGMM ttoo bbee rreehhaabbiilliittaatteedd iinn aa rreessiidde<span style='color:red'>en</span>nttiiaall tteenneemme<span style='color:red'>en</span>ntt iinn lliieeuu
to different parts of the plot under the CDS, a weighted average of all
ooff hhiiss ccoommmmeerrcciiaall eessttaabblliisshhmme<span style='color:red'>en</span>ntt,, sshhaallll aallssoo bbee ggoovveerrnneedd bbyy tthhee
the applicable rates shall be taken for calculating the Average Land Rate
aaffoorreessaaiidd pprroovviissiioonnss aapppplliiccaabbllee ttoo tthhee rreessiidde<span style='color:red'>en</span>nttiiaall ooccccuuppaannttss iinn tthheessee
and the Basic Ratio. Provided further that for calculation of the Basic
RReegguullaattiioonnss..
Ratio, the Land Rate (LR) and the Rate of Construction (RC) shall be
55.. TThhee lliisstt ooff eelliiggiibbllee ooccccuuppaannttss aanndd aarreeaa ooccccuuppiieedd bbyy eeaacchh ooff tthheemm sshhaallll
taken for the year<span style='color:red'> in</span> which the proposal is approved by the Authority
bbee cceerrttiiffiieedd bbyy tthhee rreessppeeccttiivvee aauutthhoorriittyy ii..ee.. MMHHAADDAA oorr MMCCGGMM..
and the said ratio shall remain unchanged even if such Scheme
66.. FFuunnggiibbllee CCoommppe<span style='color:red'>en</span>nssaattoorryy AArreeaa aass aapppplliiccaabbllee oonn tthhee aarreeaa ttoo bbee hhaannddeedd
undergoes any revision or modification subsequently during its course
oovveerr ttoo MMHHAADDAA//MMCCGGMM sshhaallll nnoott bbee aalllloowweedd ttoo bbee uuttiilliizzeedd oonn ssaallee
of completion.
ccoommppoonne<span style='color:red'>en</span>ntt.. NNoo pprreemmiiuumm sshhaallll bbee cchhaarrggeedd oonn tthhee ffuunnggiibbllee
ccoommppe<span style='color:red'>en</span>nssaattoorryy aarreeaa iinn rreessppeecctt ooff aarreeaa ttoo bbee hhaannddeedd oovveerr ttoo
B. For redevelopment with tripartite agreement as per 2<span style='color:red'>(</span>b) above:
MMHHAADDAA//MMCCGGMM aanndd sshhaallll bbee eexxcclluussiivvee ooff tthhee FFuunnggiibbllee ccoommppe<span style='color:red'>en</span>nssaattoorryy
When there is no developer coming forward for redevelopment of the
BBUUAA iiff aavvaaiilleedd..
building as per above regulation 7<span style='color:red'>(</span>A) and at least 51% or the occupants
PPrroovviiddeedd tthhaatt aatt tthhee ooppttiioonn ooff tthhee OOwwnneerr//DDeevveellooppeerr,, tthhee aarreeaa eeqquuiivvaalleenntt
of at least 5 building adjacent at one location requested
ttoo tthhee mmaarrkkeett vvaalluuee ((aass ppeerr AASSRR ooff tthhaatt yyeeaarr)) ooff aarreeaa aaddmmiissssiibbllee aass ppeerr
MHADA/MCGM to undertake redevelopment by appointing private
tthhee pprreessccrriibbeedd ppeerrcce<span style='color:red'>en</span>nttaaggee ooff BBUUAA ttoo MMHHAADDAA//MMCCGGMM ccaann bbee mmaaddee
developer, then<span style='color:red'> in</span> such case MHADA/MCGM may undertake the
aavvaaiillaabbllee wwiitthhiinn tthhee ssaammee oorr aaddjjooiinniinngg mmuunniicciippaall wwaarrdd ooff MMCCGGMM..
redevelopment of cluster of all such buildings belongings to the
330066 DCPR 2034 307 DCPR 2034


In [34]:
final_template = f"""
You are an legal expert assistant providing detailed and accurate information. The user has asked the following question:
**Question:** {query}

Below is relevant information retrieved from various sources:
**Retrieved Information:**
{hybrid_results}

**Instruction:** Use only the query-relevant content from the retrieved information to answer the question. Focus on providing a comprehensive, informative response based solely on the given data. If any conflicting details are present, prioritize the most reliable and consistent information.
"""
final_template

"\nYou are an legal expert assistant providing detailed and accurate information. The user has asked the following question:\n**Question:** Incentive in 33(7)\n\nBelow is relevant information retrieved from various sources:\n**Retrieved Information:**\n['Original Notifications shall\\nbe referred for perspicuity\\n33(7)(B) Additional FSI for Redevelopment of existing\\nresidential housing societies excluding buildings covered\\nunder regulation 33(7) and 33(7)(A):\\n1) In case of redevelopment of existing residential housing societies\\nexcluding buildings covered under regulation 33(7) and 33(7)(A)\\nproposed by Housing societies/landlords or through their proponents\\nwhere existing members are proposed to be re- accommodated on the\\nsame plot, incentive additional BUA to the extent of 15% of existing\\nBUA or 10 sq. m per tenement whichever is more shall be permissible\\nwithout premium.\\nProvided further that if the existing authorized BUA and incentive\\nthereon as stated above 

In [35]:
from groq import Groq
client = Groq(
    api_key='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' ,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": final_template,
        }
    ],
    model="llama-3.1-70b-versatile",
)
print(chat_completion.choices[0].message.content)

Incentive in 33(7) refers to the incentive for redevelopment of existing residential housing societies under Regulation 33(7) of the Mumbai Development Control and Promotion Regulations 2034 (DCPR 2034).

According to the retrieved information, the incentive for redevelopment under Regulation 33(7) is as follows:

- In case of redevelopment of existing residential housing societies, where existing members are proposed to be re-accommodated on the same plot, an incentive additional BUA (Built-up Area) to the extent of 15% of existing BUA or 10 sq.m per tenement, whichever is more, shall be permissible without premium.

- If the existing authorized BUA and incentive thereon as stated above is less than the permissible FSI (Floor Space Index) as per Regulation 30(A)(1), then the society may avail of ‘Additional FSI on payment of premium/TDR’ up to the limit of permissible FSI.

- If staircase, lift & lift lobby areas are claimed free of FSI by charging premium as per then prevailing Regul